To do:  
- allow `--field` to accept `File` instead of `Str` and reformat command line argument - DONE
- add input options for correcting slice-to-volume movement - DONE
    - `--mporder`, `--slspec`, `--json`, `--s2v_niter`, `--s2v_lambda`, `--s2v_interp`
- add multi-band factor options - DONE
    - `--mb`, `--mb_offs`
- add additional enumerator options for modeling eddy current (movement)
- update default options where possible so they are captured in `command.txt` - DONE
- add additional options for classifying outliers - DONE
    - `--ol_nstd`, `--ol_nvox`, `--ol_type`, `--ol_pos`, `--ol_sqr`
- add additional options for correcting susceptibility-by-movement interactions - DONE
    - `--estimate_move_by_susceptibility`, `--mbs_niter`, `--mbs_lambda`, `--mbs_ksp`, `--dont_sep_offs_move`
- add new outputs from FSL 5.0.10+

`--slm`: for data with 60 directions or more and sampled on the whole sphere, use `none`; else, may use `linear`

`--fwhm` is set to 0 by default; can also provide list that is the same length as `--niter`  
current spec only allows type float

In [ ]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import fsl, utility as niu
from numba import cuda

In [ ]:
# facilitate workflow based on topup, fieldmap, no fieldmap or synthetic b0 inputs

In [ ]:
def init_eddy_wf(ignore):
    wf = pe.Workflow(name="eddy_topup_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", 
                                                      "dwi_meta", 
                                                      "bvec_file", 
                                                      "bval_file", 
                                                      "ap_file", 
                                                      "pa_file",
                                                      "magnitude_file",
                                                      "fieldmap_file"]), 
                        name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    def gen_index(in_file):
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_index.txt",
            newpath=os.path.abspath("."),
            use_ext=False)
        
        vols = nib.load(in_file, mmap=NUMPY_MMAP).get_data().shape[-1]
        index_lines = np.ones((vols,))
        index_lines_reshape = index_lines.reshape(1, index_lines.shape[0])
        np.savetxt(out_file, index_lines_reshape, fmt="%i")
        return out_file

    gen_idx = pe.Node(
        niu.Function(
            input_names=["in_file"],
            output_names=["out_file"],
            function=gen_index),
        name="gen_index")

    # (ref lines PE - 1) * effective echo spacing (not derived vendor reported echo spacing)
    # 1/[BWPPPE * ReconMatrixPE]
    # EffectiveEchoSpacing * (ReconMatrixPE - 1)
    # 0.000279998 * (128 - 1)  
    def gen_acqparams(in_file, metadata):
        import os
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_acqparams.txt",
            newpath=os.path.abspath("."),
            use_ext=False)

        acq_param_dict = {
            "j": "0 1 0 %.7f",
            "j-": "0 -1 0 %.7f",
            "i": "1 0 0 %.7f",
            "i-": "-1 0 0 %.7f",
            "k": "0 0 1 %.7f",
            "k-": "0 0 -1 %.7f"}

        pe_dir = metadata.get("PhaseEncodingDirection")

        total_readout = metadata.get("TotalReadoutTime")
        
        acq_param_lines = acq_param_dict[pe_dir] % total_readout
        
        if pe_dir[0] == pe_dir:
            opposite_pe_dir = "%s-" % pe_dir
        else:
            opposite_pe_dir = pe_dir[0]

        acq_param_lines = '\n'.join((acq_param_lines, acq_param_dict[opposite_pe_dir] % total_readout))

        with open(out_file, "w") as f:
            f.write(acq_param_lines)

        return out_file

    acqp = pe.Node(
        niu.Function(
            input_names=["in_file", "metadata"],
            output_names=["out_file"],
            function=gen_acqparams,
        ),
        name="acqp",
    )

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):
        """
        A function that averages the *b0* volumes from a DWI dataset.
        As current dMRI data are being acquired with all b-values > 0.0,
        the *lowb* volumes are selected by specifying the parameter b0_thresh.
        .. warning:: *b0* should be already registered (head motion artifact
        should be corrected).
        """
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix="_avg_b0", newpath=os.path.abspath(".")
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units("mm")
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=["in_dwi", "in_bval", "b0_thresh"],
            output_names=["out_file"],
            function=b0_average,
        ),
        name="b0_avg_pre",
    )

    bet_dwi0 = pe.Node(
        fsl.BET(mask=True, functional=True),
        name="bet_dwi_pre",
    )

    list_merge = pe.Node(niu.Merge(numinputs=2), name="list_merge")

    merge = pe.Node(fsl.Merge(dimension="t"), name="mergeAPPA")

    topup = pe.Node(fsl.TOPUP(), name="topup")

    ecc = pe.Node(
        fsl.Eddy(repol=True, cnr_maps=True, residuals=True),
        name="fsl_eddy",
    )

    # if nthreads not specified, do this
    import multiprocessing

    ecc.inputs.num_threads = multiprocessing.cpu_count()

    wf.connect(
        [(inputnode, gen_idx, [("dwi_file", "in_file")]),
         (inputnode, acqp, [("dwi_file", "in_file"),
                            ("dwi_meta", "metadata")]),
         (inputnode, avg_b0_0, [("dwi_file", "in_dwi"),
                                ("bval_file", "in_bval")]),
         (avg_b0_0, bet_dwi0, [("out_file", "in_file")]),
         (inputnode, list_merge, [("ap_file", "in1"),
                                  ("pa_file", "in2")]),
         (list_merge, merge, [("out", "in_files")]),
         (merge, topup, [("merged_file", "in_file")]),
         (acqp, topup, [("out_file", "encoding_file")]),
         (topup, ecc, [("out_fieldcoef", "in_topup_fieldcoef"),
                       ("out_movpar", "in_topup_movpar")]),
         (acqp, ecc, [("out_file", "in_acqp")]),
         (inputnode, ecc, [("dwi_file", "in_file"),
                           ("bval_file", "in_bval"),
                           ("bvec_file", "in_bvec")]),
         (bet_dwi0, ecc, [("mask_file", "in_mask")]),
         (gen_idx, ecc, [("out_file", "in_index")])
        ]
    )
    
    return wf

In [ ]:
data_dir = os.path.abspath("../data")
layout = BIDSLayout(data_dir)

In [ ]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [ ]:
dwi_file = dwi_files[-5]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)
dwi_meta = layout.get_metadata(dwi_file)

fmaps = []
fmaps = layout.get_fieldmap(dwi_file, return_list=True)
for fmap in fmaps:
    fmap["metadata"] = layout.get_metadata(fmap[fmap["suffix"]])

ap_file = fmaps[0]['epi']
pa_file = fmaps[1]['epi']

In [ ]:
test_wf = init_eddy_topup_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.dwi_meta = dwi_meta
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file
inputspec.inputs.ap_file = ap_file
inputspec.inputs.pa_file = pa_file

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True
test_wf.config["execution"]["crashfile_format"] = "txt"

test_wf.run()

In [ ]:
dwi_files

In [11]:
dwi_file = dwi_files[0]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)
dwi_meta = layout.get_metadata(dwi_file)

fmaps = []
fmaps = layout.get_fieldmap(dwi_file, return_list=True)
for fmap in fmaps:
    fmap["metadata"] = layout.get_metadata(fmap[fmap["suffix"]])

In [ ]:
os.getcwd()

In [ ]:
test_wf = init_eddy_fmap_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.dwi_meta = dwi_meta
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file
inputspec.inputs.fmap_file = fmaps[0]['fieldmap']
inputspec.inputs.mag_file = fmaps[0]['magnitude']

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True
test_wf.config["execution"]["crashfile_format"] = "txt"

test_wf.run()

190830-11:40:27,646 nipype.workflow INFO:
	 Generated workflow graph: /mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/eddy_fmap_wf/graph.png (graph2use=colored, simple_form=True).
190830-11:40:27,651 nipype.workflow INFO:
	 Workflow eddy_fmap_wf settings: ['check', 'execution', 'logging', 'monitoring']
190830-11:40:27,764 nipype.workflow INFO:
	 Running serially.
190830-11:40:27,767 nipype.workflow INFO:
	 [Node] Setting-up "eddy_fmap_wf.radToHz" in "/mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/eddy_fmap_wf/radToHz".
190830-11:40:27,776 nipype.workflow INFO:
	 [Node] Cached "eddy_fmap_wf.radToHz" - collecting precomputed outputs
190830-11:40:27,777 nipype.workflow INFO:
	 [Node] "eddy_fmap_wf.radToHz" found cached.
190830-11:40:27,779 nipype.workflow INFO:
	 [Node] Setting-up "eddy_fmap_wf.b0_avg_pre" in "/mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/eddy_fmap_wf/b0_avg_pre".
190830-11:40:27,786 nipype.workflow INFO:
	

In [10]:
def init_eddy_fmap_wf():
    wf = pe.Workflow(name="eddy_fmap_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", 
                                                      "dwi_meta", 
                                                      "bvec_file", 
                                                      "bval_file", 
                                                      "fmap_file", 
                                                      "mag_file"]), 
                        name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    def gen_index(in_file):
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_index.txt",
            newpath=os.path.abspath("."),
            use_ext=False)
        
        vols = nib.load(in_file, mmap=NUMPY_MMAP).get_data().shape[-1]
        index_lines = np.ones((vols,))
        index_lines_reshape = index_lines.reshape(1, index_lines.shape[0])
        np.savetxt(out_file, index_lines_reshape, fmt="%i")
        return out_file

    gen_idx = pe.Node(
        niu.Function(
            input_names=["in_file"],
            output_names=["out_file"],
            function=gen_index),
        name="gen_index")

    def gen_acqparams(in_file, metadata):
        import os
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_acqparams.txt",
            newpath=os.path.abspath("."),
            use_ext=False)

        acq_param_dict = {
            "j": "0 1 0 %.7f",
            "j-": "0 -1 0 %.7f",
            "i": "1 0 0 %.7f",
            "i-": "-1 0 0 %.7f",
            "k": "0 0 1 %.7f",
            "k-": "0 0 -1 %.7f"}

        pe_dir = metadata.get("PhaseEncodingDirection")

        total_readout = metadata.get("TotalReadoutTime")
        
        acq_param_lines = acq_param_dict[pe_dir] % total_readout
        
        with open(out_file, "w") as f:
            f.write(acq_param_lines)

        return out_file

    acqp = pe.Node(
        niu.Function(
            input_names=["in_file", "metadata"],
            output_names=["out_file"],
            function=gen_acqparams,
        ),
        name="acqp",
    )

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):
        """
        A function that averages the *b0* volumes from a DWI dataset.
        As current dMRI data are being acquired with all b-values > 0.0,
        the *lowb* volumes are selected by specifying the parameter b0_thresh.
        .. warning:: *b0* should be already registered (head motion artifact
        should be corrected).
        """
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix="_avg_b0", newpath=os.path.abspath(".")
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units("mm")
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=["in_dwi", "in_bval", "b0_thresh"],
            output_names=["out_file"],
            function=b0_average,
        ),
        name="b0_avg_pre",
    )

    bet_dwi0 = pe.Node(
        fsl.BET(frac=0.3, mask=True, robust=True),
        name="bet_dwi_pre",
    )
    
    rad_to_hz = pe.Node(fsl.BinaryMaths(operation="div", operand_value=6.28), name="radToHz")
    
    mag_flirt = pe.Node(fsl.FLIRT(dof=6), name="magFlirt")
    
    fmap_flirt = pe.Node(fsl.FLIRT(apply_xfm=True), name="fmapFlirt")

    ecc = pe.Node(
        fsl.Eddy(repol=True, cnr_maps=True, residuals=True),
        name="fsl_eddy",
    )

    wf.connect(
        [(inputnode, gen_idx, [("dwi_file", "in_file")]),
         (inputnode, acqp, [("dwi_file", "in_file"),
                            ("dwi_meta", "metadata")]),
         (inputnode, avg_b0_0, [("dwi_file", "in_dwi"),
                                ("bval_file", "in_bval")]),
         (avg_b0_0, bet_dwi0, [("out_file", "in_file")]),
         (inputnode, rad_to_hz, [("fmap_file", "in_file")]),
         (inputnode, mag_flirt, [("mag_file", "in_file")]),
         (bet_dwi0, mag_flirt, [("out_file", "reference")]),
         (rad_to_hz, fmap_flirt, [("out_file", "in_file")]),
         (bet_dwi0, fmap_flirt, [("out_file", "reference")]),
         (mag_flirt, fmap_flirt, [("out_matrix_file", "in_matrix_file")]),
         (fmap_flirt, ecc, [("out_file", "field")]),
         (acqp, ecc, [("out_file", "in_acqp")]),
         (inputnode, ecc, [("dwi_file", "in_file"),
                           ("bval_file", "in_bval"),
                           ("bvec_file", "in_bvec")]),
         (bet_dwi0, ecc, [("mask_file", "in_mask")]),
         (gen_idx, ecc, [("out_file", "in_index")])
        ]
    )
    
    return wf

## Brainsuite